In [11]:
import pandas as pd

In [4]:
movies = pd.read_csv('movies.csv', encoding='utf-8')
ratings = pd.read_csv('ratings.csv', encoding='utf-8')
tags = pd.read_csv('tags.csv', encoding='utf-8')

# Opération Movie

In [36]:
movies['list'] = pd.Series(movies['genres'].str.split('|'))

In [70]:
movies.head()

,movieId,title,genres,list
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,[Comedy]


In [50]:
genre_list = [ "Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "IMAX", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western", "(no genres listed)" ]
genres = pd.DataFrame(genre_list, columns=['genres'])
genres.head()

,genres
0,Action
1,Adventure
2,Animation
3,Children
4,Comedy


In [68]:
rows = []

for idx, row in movies.iterrows():
    movie_id = row['movieId']
    genre_list_for_movie = row['list']

    for genre in genre_list_for_movie:
        try:
            rows.append({'movieId': movie_id, 'genreId': genre_list.index(genre)})
        except ValueError:
            print(f"This genre was not found: {genre}")
movie_genre = pd.DataFrame(rows)

In [72]:
movie_genre.head()
movie_genre.shape

(154170, 2)

# Opération ratings

In [73]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [74]:
ratings['rating'] = ratings['rating'].apply(lambda x: x*2).astype(int)
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,8,944249077
1,1,25,2,944250228
2,1,29,4,943230976
3,1,30,10,944249077
4,1,32,10,943228858


# Operation tags

In [75]:
tags.head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


# User

In [79]:
user_id = ratings['userId'].unique()
type(user_id)

numpy.ndarray

In [77]:
tags['userId'].unique()

array([    22,     34,     55, ..., 162274, 162278, 162279],
      shape=(15848,))

In [80]:
for idx, row in tags.iterrows():
    if row['userId'] not in user_id:
        print(row['movieId'])

Sans output, tags ne contient pas d'userId exclusif. On peut utiliser que les userId de ratings

In [81]:
users = pd.DataFrame([], columns=['userId', 'username', 'password'])
users['userId'] = user_id

In [82]:
users.head()

,userId,username,password
0,1,NaN,NaN
1,2,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN
